# 1.5. Практика

Перевести ваш спарсенный json в формат csv таблицу через pandas

Для hh.ru:

1. Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)
1. Какая средняя и медианная зарплата по группам вакансий?
1. Какая средняя и медианная зарплата по каждому региону?
1. Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?
1. Какое процентное соотношение каждого региона по вакансиям от всех вакансий?
1. Какая корреляция уровня опыта от зарплаты?
1. Сколько должностей в наборе данных?
1. Какие 10 наиболее часто встречающихся должностей?


## Подготовка данных


Загрузка данных и сохранение в csv


In [1]:
import json
import pandas as pd
from pandas import DataFrame

with open("vacancies.json", "r", encoding="utf8") as f:
    data = json.load(f)

data_list = []
for i in data["data"]:
    data_list.append(
        (i["title"], i["work expeirence"], i["salary"], i["region"]))


df = pd.DataFrame(data_list, columns=data["data"][0].keys())
df.to_csv("vacancies.csv", index=False)
df


,title,work expeirence,salary,region
0,Backend-разработчик (Python),1–3 года,от 80 000 до 170 000 руб. на руки,Бишкек
1,Разработчик C++ (релокация в Dubai),3–6 лет,от 6 000 до 8 000 USD на руки,эмират Дубай
2,Python разработчик (backend),1–3 года,з/п не указана,Екатеринбург
3,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,от 600 000 руб. на руки,Москва
4,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана
...,...,...,...,...
795,Lead Fraud Analyst,1–3 года,з/п не указана,Санкт-Петербург
796,Архитектор ИТ-решений / Solution Architect,1–3 года,з/п не указана,Москва
797,Frontend React RAIC,3–6 лет,з/п не указана,Москва
798,Senior Verification Engineer,более 6 лет,з/п не указана,Зеленоград


### Разметка данных


Поскольку уровень зарплаты представлен в нескольких валютах и имеет указанные границы (от до), то нужно привести все значения к одному значению в одной валюте.


In [2]:
from numpy import NaN

salary_df = df["salary"].str.extract(
    r"^(от )?(?P<from>[\d\s]*)?( до )?(?P<to>[\d\s]*) (?P<currency>\D{3}).*$")

salary_df["from"] = salary_df["from"].str.replace("\xa0", "")
salary_df["to"] = salary_df["to"].str.replace("\xa0", "")

salary_df.loc[salary_df['from'] == "", 'from'] = salary_df["to"]
salary_df.loc[salary_df['to'] == "", 'to'] = salary_df["from"]

salary_df["from"].astype(float)
salary_df["to"].astype(float)

salary_df.groupby("currency").count().sort_values(by="from")


,0,from,2,to
currency,,,,
KGS,1,1,0,1
бел,1,1,0,1
сум,1,1,0,1
KZT,6,6,5,6
EUR,7,7,7,7
USD,25,25,19,25
руб,209,209,115,209


In [3]:
cur_convert_dict = {
    "руб": 1,
    "USD": 63.4917,
    "EUR": 62.5903,
    "KZT": 0.132858,
    "сум": 0.000572239,
    "бел": 24.7011,
    "KGS": 0.759606,
}

salary_df["from"] = salary_df.apply(lambda row: float(row["from"]) * cur_convert_dict[row["currency"]]
                                    if row["from"] is not NaN else pd.NA, axis=1)
salary_df["to"] = salary_df.apply(lambda row: float(row["to"]) * cur_convert_dict[row["currency"]]
                                  if row["to"] is not NaN else pd.NA, axis=1)

salary_df[["from", "to"]]


,from,to
0,80000.0,170000.0
1,380950.2,507933.6
2,<NA>,<NA>
3,600000.0,600000.0
4,26571.6,199287.0
...,...,...
795,NaN,NaN
796,NaN,NaN
797,NaN,NaN
798,NaN,NaN


In [4]:
salary_df["mean"] = salary_df.apply(lambda row: (row["from"] + row["to"]) / 2
                                    if row["from"] is not NaN and row["to"] is not NaN else pd.NA, axis=1)
salary_df["mean"]


0      125000.0
1      444441.9
2          <NA>
3      600000.0
4      112929.3
         ...   
795        <NA>
796        <NA>
797        <NA>
798        <NA>
799        <NA>
Name: mean, Length: 800, dtype: object

In [5]:
df["salary"] = salary_df["mean"]
df


,title,work expeirence,salary,region
0,Backend-разработчик (Python),1–3 года,125000.0,Бишкек
1,Разработчик C++ (релокация в Dubai),3–6 лет,444441.9,эмират Дубай
2,Python разработчик (backend),1–3 года,<NA>,Екатеринбург
3,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,600000.0,Москва
4,Python Разработчик,1–3 года,112929.3,Астана
...,...,...,...,...
795,Lead Fraud Analyst,1–3 года,NaN,Санкт-Петербург
796,Архитектор ИТ-решений / Solution Architect,1–3 года,NaN,Москва
797,Frontend React RAIC,3–6 лет,NaN,Москва
798,Senior Verification Engineer,более 6 лет,NaN,Зеленоград


## 1. Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)


In [6]:
directions = {
    "Программист python": [("python", "программист"), ("python", "разработчик"), ("python", "developer"), ("phyton", "разработчик"), ("python", "engineer")],
    "Аналитик": [("аналитик"), ("analyst")],
    "QA": [("qa"), ("тестиров"), ("tester")],
    "Архитектор ПО": [("архитектор")],
    "Data Science": [("data science")],
    "Machine Learning": [("machine learning"), ("ML ")],
    "Разработчик": [("разработчик"), ("developer"), ("программист"), ('react')],
    "Руководитель": [("руководитель")],
    "Преподаватель": [("преподаватель"), ("ментор")],
    "DevOps": [("devops"), ("linux")],
}


def find_direction(row):
    for key, value in directions.items():
        for variant in value:
            counter = 0
            for word in variant:
                if word in row["title"].lower():
                    counter += 1

            if counter == len(variant):
                return key
    return "Other"


df["direction"] = df.apply(lambda row: find_direction(row), axis=1)
df.groupby("direction").count()
#df.where(df.title.str.contains("python", case=False) and (df.title.str.contains("программист", case=False) or df.title.str.contains("разработчик", case=False)))


,title,work expeirence,salary,region
direction,,,,
Data Science,7,7,2,7
DevOps,28,28,5,28
Machine Learning,1,1,0,1
Other,31,31,5,31
QA,108,108,28,108
Аналитик,99,99,42,99
Архитектор ПО,6,6,1,6
Преподаватель,14,14,5,14
Программист python,348,348,106,348


## 2. Какая средняя и медианная зарплата по группам вакансий?


In [7]:
direction_mean = df[["direction", "salary"]
                    ].dropna().groupby("direction").mean()
direction_mean.rename(columns={"salary": "mean_salary"}, inplace=True)
direction_median = df[["direction", "salary"]
                      ].dropna().groupby("direction").median()
direction_median.rename(columns={"salary": "median_salary"}, inplace=True)
pd.merge(direction_mean, direction_median, on=[
         "direction"], how='inner').sort_values(by="direction")


,mean_salary,median_salary
direction,,
Data Science,78483.500000,78483.5
DevOps,256163.190000,250000.0
Other,367093.875000,250000.0
QA,215429.871250,185000.0
Аналитик,116683.875595,83750.0
Архитектор ПО,400000.000000,400000.0
Преподаватель,84111.468000,70000.0
Программист python,177485.431453,166250.0
Разработчик,162444.370463,150000.0


## 3. Какая средняя и медианная зарплата по каждому региону?


In [8]:
salary_mean = df[["region", "salary"]].dropna().groupby("region").mean()
salary_mean.rename(columns={"salary": "mean_salary"}, inplace=True)
salary_median = df[["region", "salary"]].dropna().groupby("region").median()
salary_median.rename(columns={"salary": "median_salary"}, inplace=True)
pd.merge(salary_mean, salary_median, on=[
         "region"], how='inner').sort_values(by="region")


,mean_salary,median_salary
region,,
Алматы,143083.765000,132140.6625
Армения,223651.962500,223651.9625
Архангельск,50000.000000,50000.0000
Астана,197492.000000,183448.0500
Астрахань,40000.000000,40000.0000
Атырау,26571.600000,26571.6000
Баку,160000.000000,160000.0000
Балашиха,45000.000000,45000.0000
Батуми,190475.100000,190475.1000


## 4. Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?


In [9]:
df[["direction", "salary"]].dropna().groupby(
    "direction").mean().sort_values("salary", ascending=False).head(1)


,salary
direction,
Архитектор ПО,400000.0


## 5. Какое процентное соотношение каждого региона по вакансиям от всех вакансий?


In [10]:
region_count = df[["region", "title"]].groupby("region").count()
vac_count = df.shape[0]
region_count["title"] = region_count.apply(
    lambda row: round((row["title"]/vac_count) * 100, 2), axis=1)
region_count.rename(columns={"title": "vac %"}, inplace=True)
region_count.sort_values(by="vac %", ascending=False)


,vac %
region,
Москва,49.62
Санкт-Петербург,13.63
Новосибирск,3.25
Минск,2.62
Алматы,2.50
...,...
Moscow,0.12
Новочеркасск,0.12
ОАЭ,0.12


## 6. Какая корреляция уровня опыта от зарплаты?


In [11]:
exp_salary = df.dropna()[["work expeirence", "salary"]]
exp_salary["work expeirence"].unique()


array(['1–3 года', '3–6 лет', 'более 6 лет', 'не требуется'], dtype=object)

In [12]:
exp_norm = {
    'не требуется': 0,
    '1–3 года': 0.33,
    '3–6 лет': 0.66,
    'более 6 лет': 1,
}
exp_salary["work expeirence"] = exp_salary.apply(
    lambda row: exp_norm[row["work expeirence"]], axis=1)


In [13]:
max_salary = exp_salary["salary"].max()

exp_salary["salary"] = exp_salary.apply(
    lambda row: row["salary"]/max_salary, axis=1)

exp_salary.corr().loc["work expeirence", "salary"]


0.547973387014007

Корреляция уровня опыта от зарплаты = 0.55 - средняя корреляция.


## 7. Сколько должностей в наборе данных?


In [14]:
df["title"].nunique()


606

## 8. Какие 10 наиболее часто встречающихся должностей?


In [15]:
s = df.groupby(["title"])["title"].count(
).sort_values(ascending=False).head(10)
s


title
Разработчик Python                                               17
Python разработчик                                               16
Программист Python                                               15
Специалист службы поддержки с техническими знаниями (Контест)    12
Senior/Lead Python-разработчик                                   10
Python Developer                                                 10
Middle Python Developer                                           9
Инженер-программист                                               7
Senior Python developer                                           7
Python-разработчик                                                7
Name: title, dtype: int64